# Homework 2 Programming Exercise

In [298]:
import pandas as pd
import numpy as np

In [299]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [300]:
print('Training data points:', len(train))
print('Test data points:', len(test))

Training data points: 7613
Test data points: 3263


There are 7613 training data points and 3263 test data points

In [301]:
len(train[train['target'] == 1]) / len(train)

0.4296597924602653

In [302]:
len(train[train['target'] == 0]) / len(train)

0.5703402075397347

\~43% of the training tweets are about real disasters and  \~57% of the training tweets are not about real disasters.

In [303]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [304]:
from sklearn.model_selection import train_test_split 

X = list(train.columns[:-1])

# Splits train.csv into training set (70%) and development set (30%)
X_train, X_dev, y_train, y_dev = train_test_split(train[X], train['target'], test_size=0.3, random_state=0)
X_train

,id,keyword,location,text
476,686,attack,#UNITE THE BLUE,@blazerfan not everyone can see ignoranceshe i...
4854,6913,mass%20murderer,NaN,White people I know you worry tirelessly about...
4270,6066,heat%20wave,NaN,Chilli heat wave Doritos never fail!
992,1441,body%20bagging,New Your,@BroseidonRex @dapurplesharpie I skimmed throu...
4475,6365,hostages,cuba,#hot C-130 specially modified to land in a st...
...,...,...,...,...
4931,7025,mayhem,"Manavadar, Gujarat",They are the real heroes... RIP Brave hearts.....
3264,4689,engulfed,USA,Car engulfed in flames backs up traffic at Par...
1653,2388,collapsed,"Alexandria, Egypt.",Great British Bake Off's back and Dorret's cho...
2607,3742,destroyed,USA,Black Eye 9: A space battle occurred at Star O...


In [305]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet 
from nltk.tag import pos_tag

# import nltk
# nltk.download()

# Lemmatize words based on part of speech (verbs, adjectives, and nouns)
def lemmatize(text):
    wnl = WordNetLemmatizer()
    word_tags = pos_tag(text.split())
    result_text = []
    for word_tag in word_tags:
        lemmatized_word = word_tag[0]
        # lemmatize verbs (e.g. ate -> eat)
        if 'VB' in word_tag[1]:
            lemmatized_word = wnl.lemmatize(word_tag[0], pos='v')
        # lemmatize adjectives (e.g. better -> good)
        elif 'JJ' in word_tag[1]:
            lemmatized_word = wnl.lemmatize(word_tag[0], pos='a')
        # lemmatize nouns (e.g. cookies -> cookie)
        elif 'NN' in word_tag[1]:
            lemmatized_word = wnl.lemmatize(word_tag[0], pos='n')
        result_text.append(lemmatized_word)
    return ' '.join(result_text)

X_train['text'] = X_train['text'].apply(lambda text: lemmatize(text))
X_dev['text'] = X_dev['text'].apply(lambda text: lemmatize(text))
X_train['text']

476     @blazerfan not everyone can see ignoranceshe b...
4854    White people I know you worry tirelessly about...
4270                 Chilli heat wave Doritos never fail!
992     @BroseidonRex @dapurplesharpie I skim through ...
4475    #hot C-130 specially modify to land in a stadi...
                              ...                        
4931    They be the real heroes... RIP Brave hearts......
3264    Car engulfed in flame back up traffic at Parle...
1653    Great British Bake Off's back and Dorret's cho...
2607    Black Eye 9: A space battle occur at Star O784...
2732    ???????????? @MikeParrActor absolutely devasta...
Name: text, Length: 5329, dtype: object

In [306]:
import re

stop_words = ['the', 'a', 'an', 'and', 'or', 'this', 'that', 'i', 'my', 'me', 'we', 'us', 'our', 'she', 'her', 
              'he', 'his', 'him', 'they', 'their', 'them', 'you', 'your', 'there', 'are', 'is', 'from', 'to',
              'will', 'can', 'cant', 'would', 'has', 'have', 'could', 'be', 'as', 'if', 'in', 'on', 'also', 'at', 
              'of', 'into', 'by', 'be', 'it', 'its', 'so', 'im', 'youre', 'theyre', 'hes', 'shes', 'were', 'was', 
              'not','but', 'no', 'never', 'with', 'really', 'do', 'for', 'about', 'what', 'how', 'who', 'just',
              'when', 'via', 'which', 'than']

def regex_stop_word(words):
    regex = r'\b'
    for i in range(len(words)):
        if i == len(words) - 1:
            regex += words[i] + r'\b'
        else:
            regex += words[i] + r'\b|\b'
    return regex


# Converts all the words to lowercase
X_train['text'] = X_train['text'].apply(lambda text: text.lower())
X_dev['text'] = X_dev['text'].apply(lambda text: text.lower())

# Removes URLs
X_train['text'] = X_train['text'].apply(lambda text: re.sub(r'http\S+', '', text))
X_dev['text'] = X_dev['text'].apply(lambda text: re.sub(r'http\S+', '', text))

# Removes user id
X_train['text'] = X_train['text'].apply(lambda text: re.sub(r'@(.*?)[\s]', ' ', text))
X_dev['text'] = X_dev['text'].apply(lambda text: re.sub(r'@(.*?)[\s]', ' ', text))

# Strips punctuations
X_train['text'] = X_train['text'].apply(lambda text: re.sub(r'[^\w\s]', '', text))
X_dev['text'] = X_dev['text'].apply(lambda text: re.sub(r'[^\w\s]', '', text))

# Strips the stop words (the, a, an, and, or)
X_train['text'] = X_train['text'].apply(lambda text: re.sub(regex_stop_word(stop_words), '', text))
X_dev['text'] = X_dev['text'].apply(lambda text: re.sub(regex_stop_word(stop_words), '', text))

In [307]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True, min_df=20)
vtz = vectorizer.fit(X_train['text'])
V_train = vtz.transform(X_train['text']).toarray()
V_dev = vtz.transform(X_dev['text']).toarray()

In [308]:
# without regularization terms

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

clf = LogisticRegression(penalty='none', max_iter=1000).fit(V_train, y_train)
y_train_predict = clf.predict(V_train)
results = metrics.classification_report(y_train, y_train_predict)
print(results)

y_dev_predict = clf.predict(V_dev)
results = metrics.classification_report(y_dev, y_dev_predict)
print(results)

              precision    recall  f1-score   support

           0       0.80      0.87      0.84      3004
           1       0.82      0.72      0.76      2325

    accuracy                           0.81      5329
   macro avg       0.81      0.80      0.80      5329
weighted avg       0.81      0.81      0.80      5329

              precision    recall  f1-score   support

           0       0.78      0.84      0.80      1338
           1       0.74      0.66      0.70       946

    accuracy                           0.76      2284
   macro avg       0.76      0.75      0.75      2284
weighted avg       0.76      0.76      0.76      2284



In [309]:
# with L1 regularization

from sklearn.linear_model import LogisticRegression

clf_l1 = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000).fit(V_train, y_train)
y_train_predict = clf_l1.predict(V_train)
results = metrics.classification_report(y_train, y_train_predict)
print(results)

y_dev_predict = clf_l1.predict(V_dev)
results = metrics.classification_report(y_dev, y_dev_predict)
print(results)

              precision    recall  f1-score   support

           0       0.79      0.89      0.84      3004
           1       0.83      0.69      0.75      2325

    accuracy                           0.80      5329
   macro avg       0.81      0.79      0.79      5329
weighted avg       0.81      0.80      0.80      5329

              precision    recall  f1-score   support

           0       0.77      0.86      0.81      1338
           1       0.76      0.64      0.70       946

    accuracy                           0.77      2284
   macro avg       0.77      0.75      0.76      2284
weighted avg       0.77      0.77      0.77      2284



In [310]:
# with L2 regularization

from sklearn.linear_model import LogisticRegression

clf_l2 = LogisticRegression(penalty='l2', solver='liblinear', max_iter=1000).fit(V_train, y_train)
y_train_predict = clf_l2.predict(V_train)
results = metrics.classification_report(y_train, y_train_predict)
print(results)

y_dev_predict = clf_l2.predict(V_dev)
results = metrics.classification_report(y_dev, y_dev_predict)
print(results)

              precision    recall  f1-score   support

           0       0.79      0.88      0.83      3004
           1       0.82      0.70      0.76      2325

    accuracy                           0.80      5329
   macro avg       0.81      0.79      0.79      5329
weighted avg       0.80      0.80      0.80      5329

              precision    recall  f1-score   support

           0       0.77      0.85      0.81      1338
           1       0.75      0.64      0.69       946

    accuracy                           0.76      2284
   macro avg       0.76      0.75      0.75      2284
weighted avg       0.76      0.76      0.76      2284



In [311]:
param_dict = dict()
words = vtz.inverse_transform(clf_l1.coef_)[0]
for i in range(len(words)):
    param_dict[words[i]] = clf_l1.coef_[0][i]

sorted_dict = sorted(param_dict.items(), key=lambda x: x[1], reverse=True)
sorted_dict

[('mh370', 3.4694560741394773),
 ('release', 3.373780702061715),
 ('explode', 3.2527213798808914),
 ('real', 3.0374653740939124),
 ('flooding', 3.0087194135864483),
 ('evacuate', 2.6082933374060087),
 ('wreck', 2.6016289977894203),
 ('40', 2.3548592558197257),
 ('put', 2.3424244354573176),
 ('dont', 2.184620896595518),
 ('survive', 2.155143329791834),
 ('fire', 2.0603762270578305),
 ('care', 1.96530368522401),
 ('hostage', 1.9520212044872187),
 ('already', 1.7614689148939677),
 ('death', 1.731410850837017),
 ('demolition', 1.7244425299182982),
 ('full', 1.711001293231249),
 ('check', 1.6741923647837886),
 ('crash', 1.6497349699812855),
 ('site', 1.6423214564716846),
 ('collapse', 1.6343353910265037),
 ('three', 1.6297269741193792),
 ('city', 1.6175649614062888),
 ('search', 1.5809707616679658),
 ('night', 1.5609944813262122),
 ('content', 1.5332496583679973),
 ('nuclear', 1.5311553240423814),
 ('horrible', 1.4903353078230757),
 ('accident', 1.488325801870249),
 ('video', 1.473503104988

In [312]:
Bernoulli

NameError: name 'Bernoulli' is not defined